## Load Data

In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
path = 'json/'
filename = [] #json資料夾下所有檔案名稱(YYYY-MM-DD.json)
DF_list = []  #YYYY-MM-DD

for i in os.listdir(path):
    filename.append(i)
    DF_list.append(i[:-5])

In [3]:
day = input()
day_index = DF_list.index(day)

2018-10-02


In [4]:
#------------Stored testing data---------------
files_recent = {}
fr_list = []
count = 5

for i in range(day_index-count,day_index):
    fr_list.append(filename[i][:-5])
    with open(path+filename[i]) as f:
        symbol_f = json.load(f)
    files_recent[filename[i][:-5]] = symbol_f
    f.close

print(fr_list)

['2018-09-25', '2018-09-26', '2018-09-27', '2018-09-28', '2018-10-01']


In [5]:
print(files_recent['2018-10-01']['0050'].keys())

dict_keys(['adj_close', 'close', 'high', 'low', 'open', 'volume'])


In [6]:
Stock_sym = [sym for sym in files_recent[fr_list[-1]].keys()]
Stock_sym.remove('id')
New_sym = []
Stock_feature = []
Close = []
Open = []
Volume = []
C = []
O = []

for symbol in Stock_sym:
    tmp = []
    for i in range(0,count):
        if symbol not in files_recent[fr_list[-1-i]].keys():
            break
        for attr in ['adj_close','close','high','low','open','volume']: #['adj_close','close','high','low','open','volume']:
            if files_recent[fr_list[-1-i]][symbol][attr] == None or files_recent[fr_list[-1-i]][symbol][attr] == 'NULL':
                break
            if attr == 'close':
                C.append(float(files_recent[fr_list[-1-i]][symbol]['close']))
            if attr == 'open':
                O.append(float(files_recent[fr_list[-1-i]][symbol]['open']))
            if attr != 'close':
                tmp.append(float(files_recent[fr_list[-1-i]][symbol][attr]))
                
    if len(tmp) != 25:
        continue
    Close.append(C)
    Open.append(O)
    Stock_feature.append(tmp)
    New_sym.append(symbol)

In [ ]:
New_sym = numpy.array(New_sym)
recent_days = numpy.array(Stock_feature)
stock_close = numpy.array(Close)
stock_open = numpy.array(Open)

numpy.save('../output/recent_days', recent_days)
numpy.save('../output/close_today', close_today)

x_predicted = numpy.load('../output/recent_days.npy')
y_today = numpy.load('../output/close_today.npy')

## 取一個月的時間作為Trainning Data

In [7]:
#-----------Stored trainning data--------------
fro = filename.index('2017-06-01.json')
to = filename.index('2017-09-30.json')

DF_list = [filename[d][:-5] for d in range(fro,to+1)]
DF = {}
for i in range(fro,to+1):
    with open('json/'+filename[i]) as f:
        symbol_f = json.load(f)
    DF[filename[i][:-5]] = symbol_f
    f.close()

In [ ]:
stock_feature = []
close_td = []
O_td = []
C_td = []
Open_td = []
Close_td = []
new_sym = []

for i in range(5, len(DF_list)-1):#i自5開始
    stock_list = DF[DF_list[i]].keys()#stock_list陣列存的是公司的編號
    stock_list.remove('id')
    

    for stock in stock_list:
        
        if stock not in DF[DF_list[i+1]].keys():#stock_list裡的公司編號stock不在dictionary content裡的話，就重新for迴圈
            continue
            
        tmp = []   
        for j in range(0, 5):#把所取第一天之後五天的資料建成30筆資料的陣列，依序從第六天、第五天...第二天排列。假如我取2017-10-06開始依序
            #是2017-10-11、2017-10-12、2017-10-13、2017-10-16、2017-10-17、2017-10-18共7天(至少要取7天)，則30筆資料不包刮2017-10-06的
            if stock not in DF[DF_list[i-j]].keys():
                break
            for attr in ['adj_close','close','high','low','open','volume']:
                if DF[DF_list[i-j]][stock][attr] == None or DF[DF_list[i-j]][stock][attr] == 'NULL':
                    break
                if attr == 'close':
                    C_td.append(float(DF[DF_list[-1-i]][stock]['close']))
                if attr == 'open':
                    O_td.append(float(DF[DF_list[-1-i]][stock]['open']))
                if attr != 'close':
                    tmp.append(float(DF[DF_list[-1-i]][stock][attr]))
                    
                tmp.append(float(DF[DF_list[i-j]][stock][attr]))
            
            
        if len(tmp) != 25:
            continue
        Close_td.append(C_td)
        Open_td.append(O_td)
        stock_feature.append(tmp)
        new_sym.append(stock)
            
        if DF[DF_list[i+1]][stock]['close'] != 'NULL' and DF[DF_list[i+1]][stock]['close'] != None:
            #ans = 1 if float(content[day_list[i+1]][stock]['close']) - float(content[day_list[i]][stock]['close']) > 0 else 0
            ans = float(content[day_list[i+1]][stock]['close'])
            
            answer.append(ans)#answer是第n+1天的收盤價
            stock_feature.append(tmp)#stock_feature是第n天以前5天的所有資料        


In [ ]:
for symbol in Stock_sym:
    tmp = []
    for i in range(0,count):
        if symbol not in files_recent[fr_list[-1-i]].keys():
            break
        for attr in ['adj_close','close','high','low','open','volume']: #['adj_close','close','high','low','open','volume']:
            if files_recent[fr_list[-1-i]][symbol][attr] == None or files_recent[fr_list[-1-i]][symbol][attr] == 'NULL':
                break
            if attr == 'close':
                C.append(float(files_recent[fr_list[-1-i]][symbol]['close']))
            if attr == 'open':
                O.append(float(files_recent[fr_list[-1-i]][symbol]['open']))
            if attr != 'close':
                tmp.append(float(files_recent[fr_list[-1-i]][symbol][attr]))
                
    if len(tmp) != 25:
        continue
    Close.append(C)
    Open.append(O)
    Stock_feature.append(tmp)
    New_sym.append(symbol)